## L’API OpenAI

### Installation des dépendences

#### Gestion de l'installation

Selon le contexte, l'installation des dépendances diffère.

* Via Google Colab, installation directement depuis le notebook via `%pip install mon-package -q`. Attention, installation à refaire dans chaque nouveau notebook

* En local, le plus simple est d'utiliser un environnement virtuel python conjointement avec un fichier `requirements.txt`
``` bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
```


#### Gestion des clés d'API

Pour la clé d'api OpenAI, il est indispensable de la sécuriser. Pour cela plusieurs possibilités selon le contexte:

* Via Google Colab: créez un secret Google Colab nommé `OPENAI_API_KEY` (menu de gauche) puis utiliser `google.colab.userdata.get` pour la récupérer

* En local: créez un fichier `.env`, déclarez une paire clé-valeur `OPENAI_API_KEY=votre_clé` puis utiliser `python-dotenv` pour la charger dans les variables d'environnement

Quoi qu'il arrive ne **JAMAIS** partager ou commiter vos clés d'api.

En dehors de la gestion de la clé, nous installons le wrapper de l'api OpenAI.

In [ ]:
#Décommenter sur Google Colab
#%pip install openai -q
#%pip install python-dot-env -q

In [ ]:
# Décommenter pour Google Colab
#from google.colab import userdata
#api_key=userdata.get('OPENAI_API_KEY')

#Décommenter pour Jupyter Notebook
from dotenv import load_dotenv
from os import getenv
load_dotenv()
api_key=getenv('OPENAI_API_KEY')

### Requête de complétion

* temperature - entre 0 et 1. Une valeur haute, comme 0.8 entraînera une complétion plus aléatoire. Alors qu'une valeur faible, comme 0.2 donnera une complétion plus déterminste
* max_completion_tokens - Permet de limiter le nombre de token de la complétion, pour pouvoir par exemple contrôler le coût de la génération.

#### Endpoint principal - Responses API

API principale pour manipuler un modèle OpenAI.

> [Responses API reference](https://platform.openai.com/docs/api-reference/responses)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=api_key)
response = client.responses.create(
    model="gpt-4o-mini",
    instructions="You are a coding assistant that talks like a pirate.",
    input="How do I check if a Python object is an instance of a class?",
)
print(response)

La réponse contient d'autres infos en plus de la réponse du LLM. Notamment les stats d'usage pour l'appel (nombre de token du prompt input et de la complétion).

In [ ]:
print("Response content :\n", response.output_text)
print("\n\n")
print("Response usage :\n", response.usage)

#### Endpoint legacy - Chat completions API

API *legacy* pour manipuler un modèle OpenAI (supporté indéfiniment).

> [Chat completions API reference](https://platform.openai.com/docs/api-reference/chat)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a coding assistant that talks like a pirate.",
        },
        {
            "role": "user",
            "content": "How do I check if a Python object is an instance of a class?",
        }
    ],
    model="gpt-4o-mini"
)

print(chat_completion)

La réponse contient d'autres infos en plus de la réponse du LLM. Notamment les stats d'usage pour l'appel (nombre de token du prompt input et de la complétion).

In [ ]:

print("Response content :\n", chat_completion.choices[0].message.content)
print("\n\n")
print("Response usage :\n", chat_completion.usage)

### Requête de complétion multimodal - Vision

Les endpoints Responses supporte également les inputs mutli-modaux (texte + image). 

* Via l'url de l'image:

In [ ]:
prompt = "What is in this image?"
img_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/2023_06_08_Raccoon1.jpg/1599px-2023_06_08_Raccoon1.jpg"

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"{img_url}"},
            ],
        }
    ],
)

print("Response content :\n", response.output_text)
print("\n\n")
print("Response usage :\n", response.usage)

* Ou via son contenu en base 64

In [ ]:
import base64

prompt = "What is in this image?"
with open("media/mcp.png", "rb") as image_file:
    b64_image = base64.b64encode(image_file.read()).decode("utf-8")

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image}"},
            ],
        }
    ],
)
print("Response content :\n", response.output_text)
print("\n\n")
print("Response usage :\n", response.usage)

### Streaming

L'api supporte le streaming *out-of-the-box*. La complétion est streamée, token par token, au fur et à mesure.

In [ ]:
stream = client.responses.create(
    model="gpt-4o",
    input="Write a one-sentence bedtime story about a unicorn.",
    stream=True,
)

for event in stream:
    print(event)

## Cas pratique :

1. Créer des requêtes simples vers un modèle de langage (LLM) via l’API.
2. Analyser les résultats générés en fonction des prompts et comprendre comment le modèle traite l'information.
3. Expérimenter différents types de prompts pour explorer les capacités du modèle et voir comment les réponses varient.

### Exercice 1 : Prompts descriptifs
Envoyez un prompt pour obtenir une explication simple d'un concept technique.
prompt = "Explique-moi le fonctionnement d'un moteur à combustion interne."

### Exercice 2: Questions ouvertes vs fermées
Posez des questions ouvertes et des questions fermées pour voir comment le modèle réagit.
prompt = "Quels sont les défis actuels du changement climatique ?"
prompt = "Le changement climatique affecte-t-il la montée du niveau de la mer ?"

### Exercice 3: Génération créative
Essayez de jouer avec la température (ex. 0.3 vs 0.9) et observez les différences :
prompt = "En 3 phrases, raconte une courte histoire où un robot devient ami avec un humain."